In [2]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.applications.vgg16 import VGG16

from keras.models import Sequential
from keras.layers import Dense, Flatten, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model

Modelo valor

In [3]:
# Charge model VGC16 with imagenet weights
model_vgc = VGG16(weights='imagenet', include_top=False, input_shape=(224, 150, 3))
for(layer) in model_vgc.layers:
    layer.trainable = False

x = model_vgc.output
x = Flatten()(x)
x = Dense(2048, activation='relu')(x)
x = Dense(52, activation='softmax')(x)  

model = Model(inputs=model_vgc.input, outputs=x)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


2023-05-01 12:57:12.097026: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 75, 64)       0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 75, 128)      73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 75, 128)      147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 37, 128)       0     

In [4]:
folder = 'images/training/labeled/'
from keras.applications.vgg16 import preprocess_input

# Data generator
datagen = ImageDataGenerator(preprocessing_function=preprocess_input, horizontal_flip=True, validation_split=0.2)
train_generator = datagen.flow_from_directory(folder, target_size=(224, 150), batch_size=52, class_mode='categorical', subset='training')
test_generator = datagen.flow_from_directory(folder, target_size=(224, 150), batch_size=52, class_mode='categorical', subset='validation')

print(test_generator.class_indices)


Found 2080 images belonging to 52 classes.
Found 520 images belonging to 52 classes.
{'C10': 0, 'C2': 1, 'C3': 2, 'C4': 3, 'C5': 4, 'C6': 5, 'C7': 6, 'C8': 7, 'C9': 8, 'CA': 9, 'CJ': 10, 'CK': 11, 'CQ': 12, 'D10': 13, 'D2': 14, 'D3': 15, 'D4': 16, 'D5': 17, 'D6': 18, 'D7': 19, 'D8': 20, 'D9': 21, 'DA': 22, 'DJ': 23, 'DK': 24, 'DQ': 25, 'H10': 26, 'H2': 27, 'H3': 28, 'H4': 29, 'H5': 30, 'H6': 31, 'H7': 32, 'H8': 33, 'H9': 34, 'HA': 35, 'HJ': 36, 'HK': 37, 'HQ': 38, 'S10': 39, 'S2': 40, 'S3': 41, 'S4': 42, 'S5': 43, 'S6': 44, 'S7': 45, 'S8': 46, 'S9': 47, 'SA': 48, 'SJ': 49, 'SK': 50, 'SQ': 51}


In [5]:
history = model.fit_generator(train_generator, epochs=10, validation_data=test_generator)

/var/folders/vb/vzpmbxtd0j3gyd335m8bdw_h0000gn/T/ipykernel_14465/3671890123.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_generator, epochs=10, validation_data=test_generator)


Epoch 1/10
40/40 [==============================] - 233s 6s/step - loss: 24.1623 - accuracy: 0.6442 - val_loss: 0.8988 - val_accuracy: 0.9404
Epoch 2/10
40/40 [==============================] - 231s 6s/step - loss: 0.5489 - accuracy: 0.9678 - val_loss: 0.4100 - val_accuracy: 0.9788
Epoch 3/10
40/40 [==============================] - 232s 6s/step - loss: 0.2651 - accuracy: 0.9846 - val_loss: 0.2555 - val_accuracy: 0.9827
Epoch 4/10
40/40 [==============================] - 307s 8s/step - loss: 0.1351 - accuracy: 0.9923 - val_loss: 0.4110 - val_accuracy: 0.9750
Epoch 5/10
40/40 [==============================] - 271s 7s/step - loss: 0.3004 - accuracy: 0.9865 - val_loss: 0.3127 - val_accuracy: 0.9808
Epoch 6/10
40/40 [==============================] - 1694s 43s/step - loss: 0.0856 - accuracy: 0.9937 - val_loss: 0.7966 - val_accuracy: 0.9596
Epoch 7/10
40/40 [==============================] - 230s 6s/step - loss: 0.3873 - accuracy: 0.9832 - val_loss: 0.2058 - val_accuracy: 0.9750
Epoch 8/10

In [6]:
model.save('model_fulldeck.h5')

In [21]:
def decode_predictions_custom(preds, top=3, class_list=None):
    if class_list is None:
        class_list = ['C10', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'CA', 'CJ', 'CK', 'CQ', 'D10', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'DA', 'DJ', 'DK', 'DQ', 'H10', 'H2', 'H3', 'H4', 'H5', 'H6', 'H7', 'H8', 'H9', 'HA', 'HJ', 'HK', 'HQ', 'S10', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7', 'S8', 'S9', 'SA', 'SJ', 'SK', 'SQ']
        
    results = []
    for pred in preds:
        top_indices = pred.argsort()[-top:][::-1]
        result = [(class_list[i], pred[i]*100.0) for i in top_indices]
        results.append(result)
    return results

In [39]:
import cv2
import numpy as np

from tensorflow.keras.preprocessing import image

model = keras.models.load_model('model_fulldeck.h5')

var = cv2.imread('Card 4.png')
var_resized = cv2.resize(var, (150, 224))

var_resized = image.img_to_array(var_resized)
var_resized = np.expand_dims(var_resized, axis=0)
var_resized = tf.keras.applications.imagenet_utils.preprocess_input(var_resized)
prediction = model.predict(var_resized)

decoded = decode_predictions_custom(prediction)
print(decoded)


1/1 [==============================] - 0s 238ms/step
[[('D9', 100.0), ('D8', 4.375835902393349e-12), ('D5', 2.922570188075124e-14)]]
